# Notebook 1: MED-PC Extracting the Recording Data and Metadata

# Importing the Python Libraries

In [1]:
import sys
import glob
from collections import defaultdict
import os
from datetime import datetime

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from medpc2excel.medpc_read import medpc_read
from moviepy.editor import *

In [3]:
# setting path
sys.path.append('../../../src')

In [4]:
# All the libraries that were created for this repository
import extract.dataframe
import processing.tone
import extract.metadata

# Getting the Metadata from all the MED-PC Recording Files

- Every MED-PC Recording file has Metadata about time, data, subject, group, experiment name, script used, and the MED-PC box number on the top of the recording file. We will first extract that information
    - Metadata can be background information or context about data(or files) that is often different from the actual data that is intended to be recorded.  
        For more information on Metadata: https://en.wikipedia.org/wiki/Metadata

In [5]:
cohort = "pilot_3"

In [6]:
# Getting all the file paths of the recording files(that happen to all end in `.txt`)
all_med_pc_file = glob.glob("./data/*.txt")

- The path of the directory that this notebook is in. This will be where relative paths will be based off of

In [7]:
current_working_directory = os.getcwd()

In [8]:
current_working_directory

'/nancy/user/riwata/projects/behavioral_dataframe_processing/results/2022_11_05_akmese_reward_training/pilot_3'

- All the other files in this directory. If you want to use a folder in here such as the `data` folder, you'd type `./data/{name_of_folder}` where you replace `{name_of_folder}` with the name of the folder without the `{}`. You will do this in the cells following the one below
    - The `./` means the path will reference the current directory that the command is being used from

In [9]:
os.listdir(current_working_directory)

['02_calculating_latencies.ipynb',
 'proc',
 'old',
 '.ipynb_checkpoints',
 '01_extracting_recording_data_and_metadata.ipynb',
 'data',
 '03_calculating_port_entry_precision.ipynb']

## NOTE: If you are using your own data, the following path must be changed to the directory where your MED-PC recording files are located(if they are not in the specied folder). It is recommended to create a sub folder in the `data` folder(that is in the same folder as this notebook) for each group of session you want to process and put the recording files in there

- Use the cell below instead if you're using your own data. You must change the cell below by clicking on the cell and then pressing the `esc` key. Then press the `Y` button on your keyboard. If you want to switch it back do the same but press the `R` button instead. This switches it from a cell that is run as if it's code, or back to a cell where it's just treated as text
    - **NOTE: If you are using a different folder, then change the path in the cell below**
    - The asterisk is called a wild card which will tell the computer to find all files/folders that match this pattern. The `*` means that any pattern of any length can be replaced with it. The `**` tells the computer to look in all folders that are in the specified folder. For more information: https://linuxhint.com/bash_wildcard_tutorial/
    - For more information on finding the path of your folder that contains the recording files: https://www.computerhope.com/issues/ch001708.htm

In [10]:
all_med_pc_file = glob.glob("./data/*.txt")

In [11]:
all_med_pc_file[:10]

['./data/2022-09-21_13h12m_Subject 6.1.txt',
 './data/2022-09-25_20h34m_Subject 6.2.txt',
 './data/2022-09-23_14h27m_Subject 6.4.txt',
 './data/2022-11-07_15h43m_Subject 6.3.txt',
 './data/2022-09-22_08h32m_Subject 6.4.txt',
 './data/2022-09-22_08h32m_Subject 6.1.txt',
 './data/2022-09-27_15h09m_Subject 6.3.txt',
 './data/2022-09-24_11h42m_Subject 6.3.txt',
 './data/2022-11-04_16h06m_Subject 6.1.txt',
 './data/2022-09-26_12h46m_Subject 6.2.txt']

# Filtering the MED-PC Files that have Pilot of Pilot in it

- Example of what the MED-PC Recording file looks like

In [12]:
with open(all_med_pc_file[0]) as f:
    lines = f.readlines()
    for line in lines[:20]:
        print(line)

File: C:\MED-PC\Data\2022-09-21_13h12m_Subject 6.1.txt







Start Date: 09/21/22

End Date: 09/21/22

Subject: 6.1

Experiment: CD1_vs_C57_Comparison

Group: Cage_6

Box: 4

Start Time: 13:12:41

End Time: 14:15:20

MSN: CD1_reward_training

A:    4399.000

D:    9000.000

E:       0.000

L:       0.000

M:       0.000

O:       0.000

T:    3660.000



- We will be extracting the first 10 or so lines that look like:

```
File: C:\MED-PC\Data\2022-05-06_12h59m_Subject 3.4 (2).txt

Start Date: 05/06/22

End Date: 05/06/22

Subject: 3.4 (2)

Experiment: Pilot of Pilot

Group: Cage 4

Box: 1

Start Time: 12:59:58

End Time: 14:02:38

MSN: levelNP_CS_reward_laserepochON1st_noshock
```

- We will just find all the lines that start with `"File", "Start Date", "End Date", "Subject", "Experiment", "Group", "Box", "Start Time", "End Time", or "MSN"`. And get the metadata from those lines. And then stop once all the metadata types have been collected

In [13]:
# This makes a nested dictionary of file paths to each individual metadata type
file_path_to_meta_data = extract.metadata.get_all_med_pc_meta_data_from_files(list_of_files=all_med_pc_file)

In [14]:
# The metadata for the first file
for key, value in file_path_to_meta_data.items():
    print("File path: {}".format(key))
    print("Metadata types and associated values: {}".format(value))
    break

File path: ./data/2022-09-21_13h12m_Subject 6.1.txt
Metadata types and associated values: {'File': 'C:\\MED-PC\\Data\\2022-09-21_13h12m_Subject 6.1.txt', 'Start Date': '09/21/22', 'End Date': '09/21/22', 'Subject': '6.1', 'Experiment': 'CD1_vs_C57_Comparison', 'Group': 'Cage_6', 'Box': '4', 'Start Time': '13:12:41', 'End Time': '14:15:20', 'MSN': 'CD1_reward_training'}


## Making a Dataframe out of the Metadata

- A Dataframe is essentially a "programmable" spreadsheet. But instead of clicking on cells, you will have to tell Python how you want to interact with the spreadsheet
    - For more information: https://realpython.com/pandas-dataframe/

In [15]:
# Turning the dictionary into a Pandas Dataframe
metadata_df = pd.DataFrame.from_dict(file_path_to_meta_data, orient="index")
# Resetting the index because currently the file path is the index 
metadata_df = metadata_df.reset_index()

In [16]:
metadata_df.head()

,index,File,Start Date,End Date,Subject,Experiment,Group,Box,Start Time,End Time,MSN
0,./data/2022-09-21_13h12m_Subject 6.1.txt,C:\MED-PC\Data\2022-09-21_13h12m_Subject 6.1.txt,09/21/22,09/21/22,6.1,CD1_vs_C57_Comparison,Cage_6,4,13:12:41,14:15:20,CD1_reward_training
1,./data/2022-09-25_20h34m_Subject 6.2.txt,C:\MED-PC\Data\2022-09-25_20h34m_Subject 6.2.txt,09/25/22,09/25/22,6.2,CD1_vs_C57_Comparison,Cage_6,3,20:34:57,21:38:58,CD1_reward_training
2,./data/2022-09-23_14h27m_Subject 6.4.txt,C:\MED-PC\Data\2022-09-23_14h27m_Subject 6.4.txt,09/23/22,09/23/22,6.4,CD1_vs_C57_Comparison,Cage_6,4,14:27:26,15:31:03,CD1_reward_training
3,./data/2022-11-07_15h43m_Subject 6.3.txt,C:\MED-PC\Data\2022-11-07_15h43m_Subject 6.3.txt,11/07/22,11/07/22,6.3,akmese_rotation,cage_6,4,15:43:29,16:44:41,CD1_reward_training
4,./data/2022-09-22_08h32m_Subject 6.4.txt,C:\MED-PC\Data\2022-09-22_08h32m_Subject 6.4.txt,09/22/22,09/22/22,6.4,CD1_vs_C57_Comparison,Cage_6,2,08:32:57,10:01:39,CD1_reward_training


In [17]:
metadata_df.tail()

,index,File,Start Date,End Date,Subject,Experiment,Group,Box,Start Time,End Time,MSN
35,./data/2022-09-27_15h09m_Subject 6.2.txt,C:\MED-PC\Data\2022-09-27_15h09m_Subject 6.2.txt,09/27/22,09/27/22,6.2,CD1_vs_C57_Comparison,Cage_6,2,15:09:17,16:10:23,CD1_reward_training
36,./data/2022-09-26_12h46m_Subject 6.3.txt,C:\MED-PC\Data\2022-09-26_12h46m_Subject 6.3.txt,09/26/22,09/26/22,6.3,CD1_vs_C57_Comparison,Cage_6,4,12:46:18,13:50:05,CD1_reward_training
37,./data/2022-11-04_16h06m_Subject 6.3.txt,C:\MED-PC\Data\2022-11-04_16h06m_Subject 6.3.txt,11/04/22,11/04/22,6.3,akmese_rotation,cage_6,2,16:06:55,17:08:18,CD1_reward_training
38,./data/2022-09-24_11h42m_Subject 6.4.txt,C:\MED-PC\Data\2022-09-24_11h42m_Subject 6.4.txt,09/24/22,09/24/22,6.4,CD1_vs_C57_Comparison,Cage_6,3,11:42:14,12:46:37,CD1_reward_training
39,./data/2022-11-07_15h43m_Subject 6.1.txt,C:\MED-PC\Data\2022-11-07_15h43m_Subject 6.1.txt,11/07/22,11/07/22,6.1,akmese_rotation,cage_6,2,15:43:29,16:44:41,CD1_reward_training


- Getting the numbers out of the column that contains the cage information

In [18]:
import re

In [19]:
metadata_df = metadata_df.dropna(subset="Group")

In [20]:
metadata_df = metadata_df[metadata_df["Group"] != ""]

In [21]:
metadata_df["cage"] = metadata_df["Group"].apply(lambda x: re.findall(r'\d+', x)[0])

In [22]:
metadata_df["cage"].unique()

array(['6'], dtype=object)

In [23]:
metadata_df["cage"].head()

0    6
1    6
2    6
3    6
4    6
Name: cage, dtype: object

- Getting the number of files that are associated with each subject

In [24]:
# How many files there are for each subject
metadata_df.groupby("Subject").count()

,index,File,Start Date,End Date,Experiment,Group,Box,Start Time,End Time,MSN,cage
Subject,,,,,,,,,,,
6.1,10,10,10,10,10,10,10,10,10,10,10
6.2,10,10,10,10,10,10,10,10,10,10,10
6.3,10,10,10,10,10,10,10,10,10,10,10
6.4,10,10,10,10,10,10,10,10,10,10,10


## Looking over the MED-PC scripts

- MED-PC has scripts(lines of code that act as the instructions for how to operate the MED-PC boxes) that contain the descriptions of what each value in the recordings are. We will extract the descriptions from the MED-PC scripts so that we can label the data points in the MED-PC recordings  
- Below is a list of all the MED-PC Scripts. We will only be looking at the first one    

In [25]:
all_medpc_scripts = glob.glob("./data/*.MPC")

In [26]:
all_medpc_scripts

['./data/CD1_reward_training.MPC', './data/CD1_reward_competition.MPC']

- Example of what the MED-PC Script looks like that was ran when recording the behaviors

In [27]:
with open(all_medpc_scripts[0]) as f:
    lines = f.readlines()
    for line in lines[:100]:
        print(line)

\INPUTS



^port = 8





\OUTPUTS

^whitenoise = 1

^tone1 = 2

^tone2 = 3

^pump = 9

^fan = 16

^toneled = 17

^peled = 18







\ARRAYS



DIM P = 20000 \Port entry time stamp array



DIM Q = 250 \US delivery time stamp array (absolute)



DIM R = 250 \US time stamp array (relative to last CS)



DIM W = 2500 \ITI values used for CS



DIM S = 2500 \CS presentation values (absolute - every time yellow light turns on)



DIM N = 20000 \Port exit time stamp array



DIM K = 250 \CS type



DIM G = 250 \controlled_stimulus_seconds computer clock time (seconds on clock every time yellow light/tone turns on)



DIM H = 250 \controlled_stimulus_minutes computer clock time (minutes on clock every time yellow light/tone turns on)



DIM I = 250 \controlled_stimulus_hours computer clock time (hours every time yellow light/tone turns on)



DIM B = 2500 \port_entry_seconds computer clock time (seconds on clock every time red light turns on)



DIM F = 2500 \port_entry_minutes computer cloc

- We will be using the comments in the MED-PC script(Everything after the `\` for each line) to create a name for the variables. By default, MED-PC uses a single letter as the name of the variable(programming object that holds some information).
    - This will use the medpc2excel library found in https://github.com/cyf203/medpc2excel
- Example of the variable names and their comments in the MED-PC script that we will get the descriptive names from:
    - MED-PC will squish the first few words to create the name for every variable

```
DIM P = 20000 \Port entry time stamp array

DIM Q = 2500 \US delivery time stamp array (absolute)

DIM R = 2500 \US time stamp array (relative to last CS)

DIM W = 2500 \ITI values used for CS

DIM S = 2500 \CS presentation values (absolute - every time light turns on)

DIM N = 20000 \Port exit time stamp array

DIM K = 2500 \CS type

DIM B = 2500 \shock intensity
```

- In the MED-PC recording files, there are values that are labelled with letters. These data type of these letters is described in the MED-PC script file that we just looked at.

In [28]:
with open(all_med_pc_file[0]) as f:
    lines = f.readlines()
    for line in lines[:20]:
        print(line)

File: C:\MED-PC\Data\2022-09-21_13h12m_Subject 6.1.txt







Start Date: 09/21/22

End Date: 09/21/22

Subject: 6.1

Experiment: CD1_vs_C57_Comparison

Group: Cage_6

Box: 4

Start Time: 13:12:41

End Time: 14:15:20

MSN: CD1_reward_training

A:    4399.000

D:    9000.000

E:       0.000

L:       0.000

M:       0.000

O:       0.000

T:    3660.000



## **NOTE: Please make sure that the corresponding `.mpc` file (aka the MED-PC script) that was ran to create the log file, is also in the same folder as the recording files. This notebook will fail to extract the data from the recording file if it is missing**

- Spreadsheet of the data from the MED-PC recording file for all the files combined into one.
- **NOTE: Each row does not correspond to the same data point. Each row represents the "n"-th data point for each category for each file. AKA, the first row is the first data point for the time the subject entered the port and the first data point for the time that the tone that is played, two seperate things. The second row is the second data point for all the categories, the third row is the third data point and so on. This repeats for all data points in a given file, then it starts over to the next file where the first row of that file is the first for all its data points.**

In [29]:
concatted_medpc_df = extract.dataframe.get_medpc_dataframe_from_list_of_files(medpc_files=all_med_pc_file)

In [30]:
concatted_medpc_df.head()

,(P)Portentry,(Q)USdelivery,(R)UStime,(W)ITIvalues,(S)CSpresentation,(N)Portexit,(K)CStype,(G)controlled_stimulus_secondscomputer,(H)controlled_stimulus_minutescomputer,(I)controlled_stimulus_hourscomputer,(B)port_entry_secondscomputer,(F)port_entry_minutescomputer,(J)port_entry_hourscomputer,date,subject,file_path
0,39.59,64.0,399.0,0.0,60.01,40.01,1.0,20.0,15.0,13.0,59.0,14.0,13.0,20220921,6.1,./data/2022-09-21_13h12m_Subject 6.1.txt
1,40.41,144.0,399.0,0.0,140.01,40.93,1.0,40.0,16.0,13.0,0.0,15.0,13.0,20220921,6.1,./data/2022-09-21_13h12m_Subject 6.1.txt
2,77.46,234.0,399.0,0.0,230.01,77.51,1.0,10.0,18.0,13.0,37.0,15.0,13.0,20220921,6.1,./data/2022-09-21_13h12m_Subject 6.1.txt
3,77.78,314.0,399.0,0.0,310.01,79.67,1.0,30.0,19.0,13.0,38.0,15.0,13.0,20220921,6.1,./data/2022-09-21_13h12m_Subject 6.1.txt
4,79.69,389.0,399.0,0.0,385.01,79.72,1.0,45.0,20.0,13.0,40.0,15.0,13.0,20220921,6.1,./data/2022-09-21_13h12m_Subject 6.1.txt


In [31]:
concatted_medpc_df.tail()

,(P)Portentry,(Q)USdelivery,(R)UStime,(W)ITIvalues,(S)CSpresentation,(N)Portexit,(K)CStype,(G)controlled_stimulus_secondscomputer,(H)controlled_stimulus_minutescomputer,(I)controlled_stimulus_hourscomputer,(B)port_entry_secondscomputer,(F)port_entry_minutescomputer,(J)port_entry_hourscomputer,date,subject,file_path
6003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.0,NaN,NaN,20221107,6.1,./data/2022-11-07_15h43m_Subject 6.1.txt
6004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,20221107,6.1,./data/2022-11-07_15h43m_Subject 6.1.txt
6005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2254.0,NaN,NaN,20221107,6.1,./data/2022-11-07_15h43m_Subject 6.1.txt
6006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,39.0,NaN,NaN,20221107,6.1,./data/2022-11-07_15h43m_Subject 6.1.txt
6007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.0,NaN,NaN,20221107,6.1,./data/2022-11-07_15h43m_Subject 6.1.txt


In [32]:
pilot_df = concatted_medpc_df[~concatted_medpc_df["subject"].str.contains(r'v', na=True)].copy()

In [33]:
pilot_df

,(P)Portentry,(Q)USdelivery,(R)UStime,(W)ITIvalues,(S)CSpresentation,(N)Portexit,(K)CStype,(G)controlled_stimulus_secondscomputer,(H)controlled_stimulus_minutescomputer,(I)controlled_stimulus_hourscomputer,(B)port_entry_secondscomputer,(F)port_entry_minutescomputer,(J)port_entry_hourscomputer,date,subject,file_path
0,39.59,64.0,399.0,0.0,60.01,40.01,1.0,20.0,15.0,13.0,59.0,14.0,13.0,20220921,6.1,./data/2022-09-21_13h12m_Subject 6.1.txt
1,40.41,144.0,399.0,0.0,140.01,40.93,1.0,40.0,16.0,13.0,0.0,15.0,13.0,20220921,6.1,./data/2022-09-21_13h12m_Subject 6.1.txt
2,77.46,234.0,399.0,0.0,230.01,77.51,1.0,10.0,18.0,13.0,37.0,15.0,13.0,20220921,6.1,./data/2022-09-21_13h12m_Subject 6.1.txt
3,77.78,314.0,399.0,0.0,310.01,79.67,1.0,30.0,19.0,13.0,38.0,15.0,13.0,20220921,6.1,./data/2022-09-21_13h12m_Subject 6.1.txt
4,79.69,389.0,399.0,0.0,385.01,79.72,1.0,45.0,20.0,13.0,40.0,15.0,13.0,20220921,6.1,./data/2022-09-21_13h12m_Subject 6.1.txt
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.0,NaN,NaN,20221107,6.1,./data/2022-11-07_15h43m_Subject 6.1.txt
6004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,20221107,6.1,./data/2022-11-07_15h43m_Subject 6.1.txt
6005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2254.0,NaN,NaN,20221107,6.1,./data/2022-11-07_15h43m_Subject 6.1.txt
6006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,39.0,NaN,NaN,20221107,6.1,./data/2022-11-07_15h43m_Subject 6.1.txt


In [34]:
pilot_df["date"].unique()

array(['20220921', '20220925', '20220923', '20221107', '20220922',
       '20220927', '20220924', '20221104', '20220926', '20220920'],
      dtype=object)

In [35]:
concatted_medpc_df = pilot_df.copy()

- Combining the recording and the metadata into one dataframe

In [36]:
recording_and_metadata_df = concatted_medpc_df.merge(metadata_df, left_on='file_path', right_on='index')

In [37]:
recording_and_metadata_df.head()

,(P)Portentry,(Q)USdelivery,(R)UStime,(W)ITIvalues,(S)CSpresentation,(N)Portexit,(K)CStype,(G)controlled_stimulus_secondscomputer,(H)controlled_stimulus_minutescomputer,(I)controlled_stimulus_hourscomputer,...,Start Date,End Date,Subject,Experiment,Group,Box,Start Time,End Time,MSN,cage
0,39.59,64.0,399.0,0.0,60.01,40.01,1.0,20.0,15.0,13.0,...,09/21/22,09/21/22,6.1,CD1_vs_C57_Comparison,Cage_6,4,13:12:41,14:15:20,CD1_reward_training,6
1,40.41,144.0,399.0,0.0,140.01,40.93,1.0,40.0,16.0,13.0,...,09/21/22,09/21/22,6.1,CD1_vs_C57_Comparison,Cage_6,4,13:12:41,14:15:20,CD1_reward_training,6
2,77.46,234.0,399.0,0.0,230.01,77.51,1.0,10.0,18.0,13.0,...,09/21/22,09/21/22,6.1,CD1_vs_C57_Comparison,Cage_6,4,13:12:41,14:15:20,CD1_reward_training,6
3,77.78,314.0,399.0,0.0,310.01,79.67,1.0,30.0,19.0,13.0,...,09/21/22,09/21/22,6.1,CD1_vs_C57_Comparison,Cage_6,4,13:12:41,14:15:20,CD1_reward_training,6
4,79.69,389.0,399.0,0.0,385.01,79.72,1.0,45.0,20.0,13.0,...,09/21/22,09/21/22,6.1,CD1_vs_C57_Comparison,Cage_6,4,13:12:41,14:15:20,CD1_reward_training,6


## Getting the cage numbers and the dates so that we can include it in the file name

- We will get the metadata from the recording files and use it to name the files we will create. This will help organize the files and make it easy to know where each file came from 

- Getting the group numbers(original cage names)

In [38]:
# removing blank spaces
group_numbers = ["_".join(number.split()) for number in recording_and_metadata_df["Group"].unique() if number]
# sorting numbers
group_numbers = sorted(group_numbers)
group_numbers_for_title = "_".join(group_numbers)

In [39]:
group_numbers_for_title

'Cage_6_cage_6'

- Getting the cage numbers

In [40]:
# removing blank spaces
cage_numbers = ["_".join(number.split()) for number in recording_and_metadata_df["cage"].unique() if number]
# sorting numbers
cage_numbers = sorted(cage_numbers)
cage_numbers_for_title = "_".join(cage_numbers)

In [41]:
cage_numbers_for_title

'6'

- Getting the dates

In [42]:
# Getting the first and last recording date to get a range
earliest_date = recording_and_metadata_df["date"].min()
latest_date = recording_and_metadata_df["date"].max()

In [43]:
earliest_date

'20220920'

In [44]:
latest_date

'20221107'

- Getting the subject names

In [45]:
recording_and_metadata_df["Subject"].unique()

array(['6.1', '6.2', '6.4', '6.3'], dtype=object)

In [46]:
# removing blank spaces
subject_numbers = ["subject-" + number for number in recording_and_metadata_df["Subject"].unique() if number]
# sorting numbers
subject_numbers = sorted(subject_numbers)
subject_numbers_for_title = "_".join(subject_numbers)

In [47]:
subject_numbers_for_title

'subject-6.1_subject-6.2_subject-6.3_subject-6.4'

- Getting the experiment name

In [48]:
experiment_names = ["_".join(name.split()) for name in recording_and_metadata_df["Experiment"].unique() if name]
# sorting experiment_names
experiment_names = sorted(experiment_names)
experiment_names_for_title = "AND".join(experiment_names)

In [49]:
experiment_names_for_title

'CD1_vs_C57_ComparisonANDakmese_rotation'

- Getting the box numbers

In [50]:
# removing blank spaces
box_numbers = ["_".join(number.split()) for number in recording_and_metadata_df["Box"].unique() if number]
# sorting numbers
box_numbers = sorted(box_numbers)
box_numbers_for_title = "_".join(box_numbers)

In [51]:
box_numbers_for_title

'1_2_3_4'

- Getting the script names

In [52]:
# removing blank spaces
script_names = ["_".join(name.split()) for name in recording_and_metadata_df["MSN"].unique() if name]
# sorting names
script_names = sorted(script_names)
script_names_for_title = "_".join(script_names)

In [53]:
script_names_for_title

'CD1_reward_training'

## Adding other useful information

- Adding the date as the number of subsequent days since the start

In [54]:
recording_and_metadata_df["date"].unique()

array(['20220921', '20220925', '20220923', '20221107', '20220922',
       '20220927', '20220924', '20221104', '20220926', '20220920'],
      dtype=object)

In [55]:
for subject in recording_and_metadata_df["subject"].unique():
    print(subject)

6.1
6.2
6.4
6.3


In [56]:
subject_to_date_to_session = defaultdict(dict)

for subject in recording_and_metadata_df["subject"].unique():
    subject_df = recording_and_metadata_df[recording_and_metadata_df["subject"] == subject]
    for index, date in enumerate(sorted(subject_df["date"].unique())):
        subject_to_date_to_session[subject][date] = index + 1

In [57]:
subject_to_date_to_session

defaultdict(dict,
            {'6.1': {'20220920': 1,
              '20220921': 2,
              '20220922': 3,
              '20220923': 4,
              '20220924': 5,
              '20220925': 6,
              '20220926': 7,
              '20220927': 8,
              '20221104': 9,
              '20221107': 10},
             '6.2': {'20220920': 1,
              '20220921': 2,
              '20220922': 3,
              '20220923': 4,
              '20220924': 5,
              '20220925': 6,
              '20220926': 7,
              '20220927': 8,
              '20221104': 9,
              '20221107': 10},
             '6.4': {'20220920': 1,
              '20220921': 2,
              '20220922': 3,
              '20220923': 4,
              '20220924': 5,
              '20220925': 6,
              '20220926': 7,
              '20220927': 8,
              '20221104': 9,
              '20221107': 10},
             '6.3': {'20220920': 1,
              '20220921': 2,
              '20220

In [58]:
recording_and_metadata_df["session"] = recording_and_metadata_df.apply(lambda x: subject_to_date_to_session[x["subject"]][x["date"]], axis=1)

In [59]:
recording_and_metadata_df.head()

,(P)Portentry,(Q)USdelivery,(R)UStime,(W)ITIvalues,(S)CSpresentation,(N)Portexit,(K)CStype,(G)controlled_stimulus_secondscomputer,(H)controlled_stimulus_minutescomputer,(I)controlled_stimulus_hourscomputer,...,End Date,Subject,Experiment,Group,Box,Start Time,End Time,MSN,cage,session
0,39.59,64.0,399.0,0.0,60.01,40.01,1.0,20.0,15.0,13.0,...,09/21/22,6.1,CD1_vs_C57_Comparison,Cage_6,4,13:12:41,14:15:20,CD1_reward_training,6,2
1,40.41,144.0,399.0,0.0,140.01,40.93,1.0,40.0,16.0,13.0,...,09/21/22,6.1,CD1_vs_C57_Comparison,Cage_6,4,13:12:41,14:15:20,CD1_reward_training,6,2
2,77.46,234.0,399.0,0.0,230.01,77.51,1.0,10.0,18.0,13.0,...,09/21/22,6.1,CD1_vs_C57_Comparison,Cage_6,4,13:12:41,14:15:20,CD1_reward_training,6,2
3,77.78,314.0,399.0,0.0,310.01,79.67,1.0,30.0,19.0,13.0,...,09/21/22,6.1,CD1_vs_C57_Comparison,Cage_6,4,13:12:41,14:15:20,CD1_reward_training,6,2
4,79.69,389.0,399.0,0.0,385.01,79.72,1.0,45.0,20.0,13.0,...,09/21/22,6.1,CD1_vs_C57_Comparison,Cage_6,4,13:12:41,14:15:20,CD1_reward_training,6,2


In [60]:
recording_and_metadata_df.tail()

,(P)Portentry,(Q)USdelivery,(R)UStime,(W)ITIvalues,(S)CSpresentation,(N)Portexit,(K)CStype,(G)controlled_stimulus_secondscomputer,(H)controlled_stimulus_minutescomputer,(I)controlled_stimulus_hourscomputer,...,End Date,Subject,Experiment,Group,Box,Start Time,End Time,MSN,cage,session
144254,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,11/07/22,6.1,akmese_rotation,cage_6,2,15:43:29,16:44:41,CD1_reward_training,6,10
144255,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,11/07/22,6.1,akmese_rotation,cage_6,2,15:43:29,16:44:41,CD1_reward_training,6,10
144256,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,11/07/22,6.1,akmese_rotation,cage_6,2,15:43:29,16:44:41,CD1_reward_training,6,10
144257,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,11/07/22,6.1,akmese_rotation,cage_6,2,15:43:29,16:44:41,CD1_reward_training,6,10
144258,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,11/07/22,6.1,akmese_rotation,cage_6,2,15:43:29,16:44:41,CD1_reward_training,6,10


In [61]:
recording_and_metadata_df.groupby(["subject", "date", "session"]).count().head(n=50)

(P)Portentry  (Q)USdelivery  (R)UStime  \
subject date     session                                           
6.1     20220920 1                 338             34         34   
        20220921 2                 604             35         35   
        20220922 3                 280             36         36   
        20220923 4                 966             40         40   
        20220924 5                1965             40         40   
        20220925 6                2257             40         40   
        20220926 7                2377             40         40   
        20220927 8                3571             40         40   
        20221104 9                1594             40         40   
        20221107 10               2254             40         40   
6.2     20220920 1                  57              7          7   
        20220921 2                 360             27         27   
        20220922 3                  44             12         12   
        20220923 4                  69             17         17   
        20220924 5                  66             11         11   
        20220925 6                3437             40         40   
        20220926 7                1193             40         40   
        20220927 8                1690             40         40   
        20221104 9                2134             40         40   
        20221107 10               2057             40         40   
6.3     20220920 1                 225             27         27   
        20220921 2                 607             38         38   
        20220922 3                 137             23         23   
        20220923 4                 127             22         22   
        20220924 5                 190             26         26   
        20220925 6                 722             40         40   
        20220926 7                3518             40         40   
        20220927 8                3622             40         40   
        20221104 9                1557             40         40   
        20221107 10               1784             40         40   
6.4     20220920 1                 551             38         38   
        20220921 2                 403             37         37   
        20220922 3                1191             38         38   
        20220923 4                 285             36         36   
        20220924 5                 171             39         39   
        20220925 6                2325             40         40   
        20220926 7                1903             40         40   
        20220927 8                4736             40         40   
        20221104 9                2107             40         40   
        20221107 10               3810             40         40   

                          (W)ITIvalues  (S)CSpresentation  (N)Portexit  \
subject date     session                                                 
6.1     20220920 1                2501                 98          338   
        20220921 2                2501                 98          604   
        20220922 3                2501                 98          280   
        20220923 4                2501                 98          966   
        20220924 5                2501                 98         1965   
        20220925 6                2501                 98         2257   
        20220926 7                2501                 98         2377   
        20220927 8                2501                 98         3571   
        20221104 9                2501                 98         1594   
        20221107 10               2501                 98         2254   
6.2     20220920 1                2501                 98           57   
        20220921 2                2501                 98          360   
        20220922 3                2501                 98           44   
        20220923 4                2501                 98  

## Adding the strain information

In [62]:
cage_to_strain = {"1": "C57", "2": "C57", "3": "C57", "4": "CD1", "5": "CD1", "6":"CD1"}

In [63]:
recording_and_metadata_df["strain"] = recording_and_metadata_df["cage"].map(cage_to_strain)

In [64]:
recording_and_metadata_df["strain"].unique()

array(['CD1'], dtype=object)

## Saving the dataframes(spreadsheets to files) with the metadata as part of the name

- Making necessary directories
    - If you want to use any of the other metadata as part of the name, you will have to swap out the variables in the `format()` and change the name of the folder to match your new name. The variable names are the word that's in front of the `=` at the last line of each cell. The `{}` are where the metadata variables will be inserted into the file name. For more information on formatting strings: https://www.w3schools.com/python/ref_string_format.asp
    - You can also just manually rename the files by replacing everything in `""` and removing the `.format()` part
- **NOTE: You may get an error that the file does not exist. If this is the case, it could be the file name is too long(an issue that may happen when using Jupyter Notebooks on Windows)**

In [65]:
output_directory = "./proc/extracted_recording_data_and_metadata/experiment_{}_cage_{}_date_{}_{}".format(experiment_names_for_title, cage_numbers_for_title, earliest_date, latest_date)

In [66]:
output_directory

'./proc/extracted_recording_data_and_metadata/experiment_CD1_vs_C57_ComparisonANDakmese_rotation_cage_6_date_20220920_20221107'

In [67]:
os.makedirs(output_directory, exist_ok=True)

In [68]:
metadata_df["cohort"] = cohort
concatted_medpc_df["cohort"] = cohort
recording_and_metadata_df["cohort"] = cohort

In [69]:
metadata_df.to_csv(os.path.join(output_directory, "metadata_{}_cage_{}_date_{}_{}.csv".format(cohort, cage_numbers_for_title, earliest_date, latest_date)))
# metadata_df.to_excel(os.path.join(output_directory, "metadata_cage_{}_date_{}_{}.xlsx".format(cage_numbers_for_title, earliest_date, latest_date)))

In [70]:
concatted_medpc_df.to_csv(os.path.join(output_directory, "MEDPC_recording_{}_cage_{}_date_{}_{}.csv".format(cohort, cage_numbers_for_title, earliest_date, latest_date)))
# concatted_medpc_df.to_excel(os.path.join(output_directory, "MEDPC_recording_cage_{}_date_{}_{}.xlsx".format(cage_numbers_for_title, earliest_date, latest_date)))

In [71]:
recording_and_metadata_df.to_csv(os.path.join(output_directory, "recording_metadata_{}_cage_{}_date_{}_{}.csv".format(cohort, cage_numbers_for_title, earliest_date, latest_date)))
# recording_and_metadata_df.to_excel(os.path.join(output_directory, "recording_and_metadata_cage_{}_date_{}_{}.xlsx".format(cage_numbers_for_title, earliest_date, latest_date)))